In [62]:
# https://medium.com/@niveditha.itengineer/learn-how-to-setup-portaudio-and-pyaudio-in-ubuntu-to-play-with-speech-recognition-8d2fff660e94

import sounddevice as sd
import numpy as np
import scipy.signal
from scipy.special import softmax
import timeit
import python_speech_features
import os
import json
#import RPi.GPIO as GPIO
#from kws_streaming.data import input_data
from kws_streaming import data
from datetime import datetime


#from tflite_runtime.interpreter import Interpreter
import tensorflow as tf


In [63]:
# Parameters
#debug_time = 1
#debug_acc = 0
#led_pin = 8
word_threshold = 0.9

sample_rate = 16000

num_channels = 1
model_path = 'models3_30k/svdf/tflite_stream_state_external/stream_state_external.tflite'


# GPIO 
#GPIO.setwarnings(False)
#GPIO.setmode(GPIO.BOARD)
#GPIO.setup(8, GPIO.OUT, initial=GPIO.LOW)

current_dir = os.getcwd()
DATA_PATH = os.path.join(current_dir, "data2/")
MODELS_PATH = current_dir

file_name='models3_30k'
train_dir = os.path.join(MODELS_PATH, file_name, 'svdf')

current_dir

'/mnt/disk0shared/bdjola/Documents/Technique/DevML/kws_streaming2/kws_streaming'

In [64]:
# below is another way of reading flags - through json
with tf.compat.v1.gfile.Open(os.path.join(train_dir, 'flags.json'), 'r') as fd:
   flags_json = json.load(fd)

class DictStruct(object):
   def __init__(self, **entries):
     self.__dict__.update(entries)

flags = DictStruct(**flags_json)

In [65]:
# prepare mapping of index to word
audio_processor = data.input_data.AudioProcessor(flags)
index_to_label = {}
# labels used for training
for word in audio_processor.word_to_index.keys():
  if audio_processor.word_to_index[word] == data.input_data.SILENCE_INDEX:
    index_to_label[audio_processor.word_to_index[word]] = data.input_data.SILENCE_LABEL
  elif audio_processor.word_to_index[word] == data.input_data.UNKNOWN_WORD_INDEX:
    index_to_label[audio_processor.word_to_index[word]] = data.input_data.UNKNOWN_WORD_LABEL
  else:
    index_to_label[audio_processor.word_to_index[word]] = word

# training labels
index_to_label

2024-02-26 19:43:17.213515: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-02-26 19:43:17.213550: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1261] Device interconnect StreamExecutor with strength 1 edge matrix:
2024-02-26 19:43:17.213558: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1267]      


{10: 'follow',
 16: 'learn',
 11: 'forward',
 2: 'backward',
 3: 'bed',
 5: 'cat',
 26: 'sheila',
 14: 'happy',
 4: 'bird',
 18: 'marvin',
 15: 'house',
 6: 'dog',
 12: 'four',
 21: 'off',
 8: 'eight',
 17: 'left',
 33: 'visual',
 22: 'on',
 13: 'go',
 23: 'one',
 7: 'down',
 25: 'seven',
 19: 'nine',
 20: 'no',
 9: 'five',
 30: 'tree',
 34: 'wow',
 29: 'three',
 32: 'up',
 31: 'two',
 36: 'zero',
 24: 'right',
 28: 'stop',
 27: 'six',
 35: 'yes',
 0: '_silence_'}

In [66]:
# Load model (interpreter)
#interpreter = Interpreter(model_path)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)

[{'name': 'input_audio', 'index': 0, 'shape': array([  1, 320], dtype=int32), 'shape_signature': array([  1, 320], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'streaming/speech_features/data_frame_1input_state', 'index': 1, 'shape': array([  1, 640], dtype=int32), 'shape_signature': array([  1, 640], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'streaming/svdf_0/depthwise_conv1d_6/depthwise_conv1d_6input_state', 'index': 2, 'shape': array([ 1,  4, 16], dtype=int32), 'shape_signature': array([ 1,  4, 16], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameter

In [67]:
inputs = []
for s in range(len(input_details)):
  inputs.append(np.zeros(input_details[s]['shape'], dtype=np.float32))

input_details[0]['shape']

array([  1, 320], dtype=int32)

In [68]:
# Decimate (filter and downsample)
def decimate(signal, old_fs, new_fs):
    
    # Check to make sure we're downsampling
    if new_fs > old_fs:
        print("Error: target sample rate higher than original")
        return signal, old_fs
    
    # We can only downsample by an integer factor
    dec_factor = old_fs / new_fs
    if not dec_factor.is_integer():
        print("Error: can only decimate by integer factor")
        return signal, old_fs

    # Do decimation
    resampled_signal = scipy.signal.decimate(signal, int(dec_factor))

    return resampled_signal, new_fs

In [69]:
sumseconds=0
sumrecords=0
import time as timelib

def sd_callback(rec, frames, time, status):   
    tstart = timelib.time()

    # audio rec shape is : (frames, channels)
    rec = np.reshape(rec, (1,-1))

    # set input audio data (by default input data at index 0)
    interpreter.set_tensor(input_details[0]['index'], rec)

    # set input states (index 1...)
    for s in range(1, len(input_details)):
        interpreter.set_tensor(input_details[s]['index'], inputs[s])

    # run inference
    interpreter.invoke()

    # get output: classification
    out_tflite = interpreter.get_tensor(output_details[0]['index'])
    #print(start / 16000.0, np.argmax(out_tflite), np.max(out_tflite))

    # get output states and set it back to input states
    # which will be fed in the next inference cycle
    for s in range(1, len(input_details)):
    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
        inputs[s] = interpreter.get_tensor(output_details[s]['index'])

    out_tflite_argmax = np.argmax(out_tflite)

    probas = softmax(out_tflite)
    proba = np.max(probas)
    if (out_tflite_argmax>1) and (proba > word_threshold):
        print(datetime.now().strftime("%d/%m/%Y %H:%M:%S"), index_to_label[out_tflite_argmax], proba)

    tend = timelib.time()
  
    sumrecords=sumrecords+1
    sumseconds=sumseconds+tend-tstart

    


In [70]:
reset_state = True


# before processing new test sequence we can reset model state
# if we reset model state then it is not real streaming mode
if reset_state:
  for s in range(len(input_details)):
    print(input_details[s]['shape'])
    inputs[s] = np.zeros(input_details[s]['shape'], dtype=np.float32)

[  1 320]
[  1 640]
[ 1  4 16]
[ 1 10 32]
[ 1 10 32]
[ 1 10 32]
[ 1 10 64]
[  1  10 128]
[  1   1 128]


In [71]:
# Start streaming from microphone
try:
    sumrecords=0
    print('channels:',num_channels, 'samplerate:',sample_rate, 'blocksize:', flags.window_stride_samples)
    with sd.InputStream(channels=num_channels,
                        samplerate=sample_rate,
                        blocksize=flags.window_stride_samples,
                        callback=sd_callback):
        while True:
            pass

except KeyboardInterrupt:
    #print('\nRecording finished. sumrecords:', sumrecords, 'sumseconds:', sumseconds, 'average:', sumseconds/sumrecords)
    print('\nRecording finished. sumrecords:')

except Exception as e:
    print('\nRecording finished. sumrecords:')

channels: 1 samplerate: 16000 blocksize: 320


From cffi callback <function _StreamBase.__init__.<locals>.callback_ptr at 0x7fa75e2413b0>:
Traceback (most recent call last):
  File "/mnt/disk0shared/bdjola/Documents/Technique/DevML/kws_streaming2/kws_streaming/.venv/lib/python3.7/site-packages/sounddevice.py", line 846, in callback_ptr
    return _wrap_callback(callback, data, frames, time, status)
  File "/mnt/disk0shared/bdjola/Documents/Technique/DevML/kws_streaming2/kws_streaming/.venv/lib/python3.7/site-packages/sounddevice.py", line 2687, in _wrap_callback
    callback(*args)
  File "/tmp/ipykernel_5624/62304934.py", line 41, in sd_callback
UnboundLocalError: local variable 'sumrecords' referenced before assignment



Recording finished. sumrecords:
